循环画山脊图：和origin上还是有点区别

In [ ]:
import os
import glob

root_dir = r'梯度结果\4-29的梯度的结果'
file_pattern = os.path.join(root_dir, '**', 'Network_txt') 
txt_Files = glob.glob(file_pattern, recursive=True)

print(txt_Files)
print(type(txt_Files))

txt数据的整理，整理成一个csv文件

In [ ]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd 

sort_order = ['VIS', 'SMN', 'DAN', 'VAN', 'LIM', 'FPN', 'DMN']
colors = ['red', 'blue', 'green', 'yellow', 'blue', 'green', 'yellow']

for folder in txt_Files:
    print(f"正在处理文件夹: {folder}")

    txt_files = [f for f in os.listdir(folder) if f.endswith('.txt') and 'g_1' in f]
    sorted_txt_files = sorted(txt_files, key=lambda x: sort_order.index(x.split('_')[3].replace('.txt', '')))

    all_data = []
    column_names = []

    for txt_file in sorted_txt_files:
        file_path = os.path.join(folder, txt_file)
        second_part = txt_file.split('_')[3].replace('.txt', '')
        column_names.append(second_part)
        # print (second_part)
        with open(file_path, 'r') as f:
            data = f.readlines()

        column_data = [line.strip() for line in data]
        column_data = [float(line.strip()) for line in column_data]
        all_data.append(column_data)

    result = pd.DataFrame(list(zip(*all_data)), columns=column_names)
    print(type(result))
    #print("列名:", column_names)

    output_file = os.path.join(folder, 'merged_result.csv')
    result.to_csv(output_file, index=False) 
    print(f"拼接结果已保存为 CSV 文件: {output_file}")

In [ ]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd 

A = 'g_3'

sort_order = ['DMN', 'FPN', 'LIM', 'VAN', 'DAN', 'SMN', 'VIS']
colors = ['red', 'blue', 'green', 'yellow', 'purple', 'orange', 'gray']

for folder in txt_Files:
    print(f"正在处理文件夹: {folder}")
    txt_files = [f for f in os.listdir(folder) if f.endswith('.txt') and A in f]
    sorted_txt_files = sorted(txt_files, key=lambda x: sort_order.index(x.split('_')[3].replace('.txt', '')))
    # print (sorted_txt_files)

    all_data = []
    column_names = []

    for txt_file in sorted_txt_files:
        file_path = os.path.join(folder, txt_file)
        second_part = txt_file.split('_')[3].replace('.txt', '') 
        column_names.append(second_part)
        # print (second_part)
        with open(file_path, 'r') as f:
            data = f.readlines()
        column_data = [line.strip() for line in data]
        column_data = [float(line.strip()) for line in column_data]
        all_data.append(column_data)

    result = pd.DataFrame(list(zip(*all_data)), columns=column_names)
    # print(type(result))
    # print("列名:", column_names)
    output_file = os.path.join(folder, f'{A}_merged_result.csv')
    result.to_csv(output_file, index=False) 
    print(f"拼接结果已保存为 CSV 文件: {output_file}")
    
    fig, ax = plt.subplots(figsize=(12, 8))
    vertical_offset = 0
    max_density = 0 
    for _, data in result.iteritems():
        current_max = np.histogram(data, bins=30, density=True)[0].max()
        max_density = max(max_density, current_max)
    scale_factor = 0.5 / max_density if max_density > 0 else 1

    for i, (system_name, data) in enumerate(result.items()):
        hist, bin_edges = np.histogram(data, bins=30, density=True)
        bin_centers = (bin_edges[:-1] + bin_edges[1:])/2

        scaled_density = hist * scale_factor + vertical_offset

        ax.plot(bin_centers, scaled_density, color=colors[i], label=system_name)
        ax.fill_between(bin_centers, vertical_offset, scaled_density, 
                        color=colors[i], alpha=0.3)

        vertical_offset += 1
        
    ax.set_yticks(np.arange(len(result.columns)))
    ax.set_yticklabels(result.columns)
    ax.set_ylim(0, len(result.columns))
    ax.set_xlabel('Gradient Values', fontsize=12)
    ax.set_title('System-based Histogram of Gradient', fontsize=14)
    ax.grid(True, linestyle='--', alpha=0.5)
    
    handles, labels = ax.get_legend_handles_labels()
    legend_order = ['VIS', 'SMN', 'DAN', 'VAN', 'LIM', 'FPN', 'DMN']
    ax.legend(ordered_handles, legend_order, loc='upper left', bbox_to_anchor=(1, 1), fontsize=10)
    output_file = os.path.join(folder, f'{A}.png') 
    plt.savefig(output_file, dpi=300)
    plt.tight_layout()

plt.close(fig)